In [1]:
#imports for langchain
# from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain_community.document_loaders import DirectoryLoader, TextLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob
import os

In [2]:
text_loader_kwargs = {'autodetect_encoding': True}
folders = [f for f in glob.glob("knowledge-base/**/*", recursive=True) if os.path.isdir(f)]

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    
    # Load .md files
    md_loader = DirectoryLoader(
        folder,
        glob="**/*.md",
        loader_cls=TextLoader,
        loader_kwargs=text_loader_kwargs,
        recursive=True
    )
    md_docs = md_loader.load()
    for doc in md_docs:
        doc.metadata["doc_type"] = doc_type
    documents.extend(md_docs)
    
    # Load .json files, cần truyền jq_schema
    json_loader = DirectoryLoader(
        folder,
        glob="**/*.json",
        loader_cls=JSONLoader,
        loader_kwargs={"jq_schema": ".questions[] | .question + \"\\n\" + .answer"},
        recursive=True
    )
    json_docs = json_loader.load()
    for doc in json_docs:
        doc.metadata["doc_type"] = doc_type
    documents.extend(json_docs)

print("Total documents loaded:", len(documents))


Total documents loaded: 45


In [3]:
documents[0]

Document(metadata={'source': 'knowledge-base/giao_thong/traffic_issue_01.md', 'doc_type': 'giao_thong'}, page_content='# Vấn đề giao thông 1\n\n## Tổng quan\nTrong thời gian gần đây, các vấn đề về giao thông đô thị ngày càng trở nên phức tạp. Nguyên nhân xuất phát từ sự gia tăng số lượng phương tiện, sự xuất hiện của các loại xe mới, và những quy định mới về việc cấm một số loại xe điện tại khu vực trung tâm.\n\n## Các loại xe mới xuất hiện\n- Xe điện cá nhân (e-bike, e-scooter)\n- Xe tự hành (autonomous vehicle)\n- Xe hybrid (lai xăng – điện)\n\n## Quy định về cấm xe điện\nTheo quy định mới, một số khu vực trung tâm sẽ cấm:\n- Xe điện hai bánh không đăng ký biển số\n- Xe điện công suất dưới 250W lưu thông trên đường quốc lộ\n\n## Người liên quan\n- Cơ quan quản lý giao thông\n- Cảnh sát giao thông\n- Người điều khiển phương tiện\n- Doanh nghiệp cung cấp dịch vụ xe điện\n\n## Mức phạt vi phạm (tham khảo)\n| Hành vi | Mức phạt |\n|---------|---------|\n| Điều khiển xe điện không biển số

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512, # Slightly smaller chunks for better retrieval
    chunk_overlap=80, # REduced overlap for performance
    separators =["\n\n", "\n", ". ", " ",  ""]
)
chunks = text_splitter.split_documents(documents)
print(f"Create {len(chunks)} chunks")

Create 65 chunks


In [5]:
chunks[7]

Document(metadata={'source': 'knowledge-base/giao_thong/traffic_issue_03.md', 'doc_type': 'giao_thong'}, page_content='## Quy định về cấm xe điện\nTheo quy định mới, một số khu vực trung tâm sẽ cấm:\n- Xe điện hai bánh không đăng ký biển số\n- Xe điện công suất dưới 250W lưu thông trên đường quốc lộ\n\n## Người liên quan\n- Cơ quan quản lý giao thông\n- Cảnh sát giao thông\n- Người điều khiển phương tiện\n- Doanh nghiệp cung cấp dịch vụ xe điện')

In [6]:
doc_type = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Các loại tài liệu đã tìm thấy: {', '.join(doc_type)}")

Các loại tài liệu đã tìm thấy: giao_thong, test, traffic_law_qa_img_files


In [7]:
for chunk in chunks:
    if 'xe' in chunk.page_content:
        print(chunk)
        print("___________")

page_content='# Vấn đề giao thông 1

## Tổng quan
Trong thời gian gần đây, các vấn đề về giao thông đô thị ngày càng trở nên phức tạp. Nguyên nhân xuất phát từ sự gia tăng số lượng phương tiện, sự xuất hiện của các loại xe mới, và những quy định mới về việc cấm một số loại xe điện tại khu vực trung tâm.

## Các loại xe mới xuất hiện
- Xe điện cá nhân (e-bike, e-scooter)
- Xe tự hành (autonomous vehicle)
- Xe hybrid (lai xăng – điện)' metadata={'source': 'knowledge-base/giao_thong/traffic_issue_01.md', 'doc_type': 'giao_thong'}
___________
page_content='## Quy định về cấm xe điện
Theo quy định mới, một số khu vực trung tâm sẽ cấm:
- Xe điện hai bánh không đăng ký biển số
- Xe điện công suất dưới 250W lưu thông trên đường quốc lộ

## Người liên quan
- Cơ quan quản lý giao thông
- Cảnh sát giao thông
- Người điều khiển phương tiện
- Doanh nghiệp cung cấp dịch vụ xe điện' metadata={'source': 'knowledge-base/giao_thong/traffic_issue_01.md', 'doc_type': 'giao_thong'}
___________
page_content

In [8]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [9]:
# đưa các đoạn văn bản (chunks) vào Vector Store, liên kết mỗi đoạn với một vector embedding

# embeddings = OpenAIEmbeddings()

# Nếu bạn muốn sử dụng embeddings miễn phí từ Hugging Face ( thay vì Open AI),
# hãy thay dòng embeddings = OpenAIEmbeddings()
# bằng:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


# import os
# os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# from sentence_transformers import SentenceTransformer
# from langchain_community.embeddings import HuggingFaceEmbeddings

# model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# embeddings = HuggingFaceEmbeddings(model=model)

# print(embeddings.embed_query("Hello world"))



/home/nguyenhaubt/anaconda3/envs/llm/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [10]:
MODEL = "PhoGPT-4B-Chat-Q4_K_M"
#đặt tên cho vecterdatabase
db_name = "vector_db"
# Kiểm tra nếu database đã tồn tại, thì xóa collection để khởi động lại từ đầu
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [11]:
#tao vector database bang chroma
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=db_name
)
#kiem tra so luong document da duoc luu vao vector store
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 65 documents


In [12]:
#lay ra bo suu tap vector tu vectorstore
collection = vectorstore._collection

#lay 1 embedding tu database
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]

#kiem tra so chieu ( so phan tu cua vector)
dimensions = len(sample_embedding)
print(f"the Vector have {dimensions:,} dimensions")

the Vector have 384 dimensions


In [13]:
sample_embedding

array([-1.03078783e-01,  6.12159520e-02,  1.92221962e-02, -9.71536860e-02,
        1.14988936e-02, -8.57933238e-03,  1.02062769e-01,  9.51457843e-02,
        4.18515168e-02, -3.04474812e-02,  1.41405970e-01, -7.58925378e-02,
        2.39442214e-02, -8.66139680e-03,  2.81840004e-02,  1.73156634e-02,
       -4.15106677e-02,  3.65777723e-02, -8.51146579e-02, -2.94041708e-02,
        2.26024222e-02,  1.93668567e-02,  2.33646389e-02,  3.31485309e-02,
       -6.42463863e-02,  2.01247185e-02,  4.81469883e-03,  6.02077283e-02,
        2.35878825e-02, -8.97502974e-02,  3.71202677e-02,  1.64740741e-01,
       -1.62388403e-02, -5.84477000e-02,  3.81844789e-02,  5.86919021e-03,
       -3.51344198e-02, -9.74030048e-02,  2.80280262e-02, -9.58121289e-03,
       -1.87254883e-02, -7.64859319e-02,  2.38899365e-02, -1.51500374e-01,
        1.00941584e-01,  4.95602377e-02, -2.47762129e-02,  1.65131111e-02,
        7.91179761e-03, -5.80023080e-02, -2.40201186e-02,  4.76934388e-02,
        1.32134771e-02,  

In [14]:
# Lấy toàn bộ vector từ database từ collection
result = collection.get(include=["embeddings", "documents", "metadatas"])

# Đưa embedding vào numpy array
vectors = np.array(result["embeddings"])

# Lưu lại văn bản
documents = result["documents"]

# Trích loại tài liệu từ metadata
doc_types = [metadata.get('doc_type', 'unknown') for metadata in result["metadatas"]]

# Mapping màu cho từng loại tài liệu
color_map = {
    'giao_thong': 'blue',
    'traffic_law_qa_img_files': 'green'
}

# Nếu loại tài liệu không có trong map thì gán 'gray'
colors = [color_map.get(t, 'gray') for t in doc_types]


In [15]:
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
reduced_vectors = tsne.fit_transform(vectors)

fig = go.Figure(
    data=[go.Scatter(
        x=reduced_vectors[:, 0],
        y=reduced_vectors[:, 1],
        mode='markers',
        marker=dict(
            size=5,
            color=colors,  # Sử dụng màu đã ánh xạ
            opacity=0.8,
            # line=dict(width=0.5, color='DarkSlateGrey'),
        ),
            text = [f"Loai: {t}<br>Van ban: {doc[:50]}..." for t, doc in zip(doc_types, documents)],
            hoverinfo='text'
        
        # text=doc_types,  # Hiển thị loại tài liệu khi di chuột
    )
])
fig.update_layout( 
    title='t-SNE Visualization of Document Embeddings',
    xaxis_title='Component 1',
    yaxis_title='Component 2',
    showlegend=False,
    width=800,
    height=600,
    margin=dict(l=10, r=20, t=40, b=10)
)

fig.show(renderer='browser')


In [16]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [17]:
llm = ChatOpenAI(
    api_key="lm",
    base_url="http://192.168.0.229:1234/v1",  # Địa chỉ của mô hình LLM
    temperature=0.4,  # Giảm nhiệt độ để tạo ra câu trả lời ổn định hơn
    model_name=MODEL,  # Sử dụng mô hình đã chọn

)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)  
memory = ConversationBufferMemory(return_messages=True)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
)

/tmp/ipykernel_293571/949354378.py:9: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [18]:
import time
def test_query_performance():
    """Test query with timings and memory usage.""" 
    query = "Xe máy có được đi vào đường cao tốc không?"
    start_time = time.time()
    result = conversation_chain.invoke({"question": query})
    end_time = time.time()

    print(f"Answer:, {result['answer']}")
    print(f"Time taken: {end_time - start_time:.2f} seconds")
    if 'source_documents' in result:
        print(f"user {len(result['source_documents'])} documents retrieved:")

In [19]:
test_query_performance

<function __main__.test_query_performance()>

In [20]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
)


In [21]:
"""
#hay cùng tìm hiểu xem điều gì được gửi phía sau hậu trường
from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(
    api_key="lm",
    base_url="http://192.168.0.229:1234/v1",  # Địa chỉ của mô hình LLM
    temperature=0.4,  # Giảm nhiệt độ để tạo ra câu trả lời ổn định hơn
    model_name=MODEL,  # Sử dụng mô hình đã chọn

)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    callbacks=[StdOutCallbackHandler()]  # In ra các bước xử lý
)
query = "Các vấn đề về giao thông đô thị ngày?"
result = conversation_chain.invoke({"question": query})
answer = result['answer']
print(f"Answer: {answer}")  
"""

'\n#hay cùng tìm hiểu xem điều gì được gửi phía sau hậu trường\nfrom langchain_core.callbacks import StdOutCallbackHandler\n\nllm = ChatOpenAI(\n    api_key="lm",\n    base_url="http://192.168.0.229:1234/v1",  # Địa chỉ của mô hình LLM\n    temperature=0.4,  # Giảm nhiệt độ để tạo ra câu trả lời ổn định hơn\n    model_name=MODEL,  # Sử dụng mô hình đã chọn\n\n)\n\nmemory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)\nretriever = vectorstore.as_retriever(search_kwargs={"k": 5})\nconversation_chain = ConversationalRetrievalChain.from_llm(\n    llm=llm,\n    retriever=retriever,\n    memory=memory,\n    callbacks=[StdOutCallbackHandler()]  # In ra các bước xử lý\n)\nquery = "Các vấn đề về giao thông đô thị ngày?"\nresult = conversation_chain.invoke({"question": query})\nanswer = result[\'answer\']\nprint(f"Answer: {answer}")  \n'

In [22]:
"""
from langchain.schema import Document
from langchain_core.retrievers import BaseRetriever
from typing import List, Dict

context = {
    "python": "Python is a programming language...",
    "ai": "Artificial Intelligence is ..."
}

# 1. Fix keyword retriever to search content, not just title
class KeywordRetriever(BaseRetriever):
    context_dict: Dict[str, str]

    def _get_relevant_documents(self, query: str, run_manager=None) -> List[Document]:
        relevant_docs = []
        for title, content in self.context_dict.items():
            if any(kw in content.lower() for kw in query.lower().split()):
                relevant_docs.append(
                    Document(page_content=content, metadata={"Source": title})
                )
        return relevant_docs

    async def _aget_relevant_documents(self, query: str, run_manager=None) -> List[Document]:
        return self._get_relevant_documents(query, run_manager)

# 2. user keyword + properly configured vector retriever
keyword_retriever = KeywordRetriever(context_dict=context)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ensemble retriever
from langchain.retrievers import EnsembleRetriever
hybrid_retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, vector_retriever],
    weights=[0.5, 0.5],
)

# 4. conversation chain
from langchain.chains import ConversationalRetrievalChain
from langchain_core.callbacks import StdOutCallbackHandler

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=hybrid_retriever,
    memory=memory,
    callbacks=[StdOutCallbackHandler()]
)

query = "Các vấn đề về giao thông đô thị ngày?"
result = conversation_chain.invoke({"question": query})
answer = result['answer']
print(f"Answer: {answer}")  

"""

'\nfrom langchain.schema import Document\nfrom langchain_core.retrievers import BaseRetriever\nfrom typing import List, Dict\n\ncontext = {\n    "python": "Python is a programming language...",\n    "ai": "Artificial Intelligence is ..."\n}\n\n# 1. Fix keyword retriever to search content, not just title\nclass KeywordRetriever(BaseRetriever):\n    context_dict: Dict[str, str]\n\n    def _get_relevant_documents(self, query: str, run_manager=None) -> List[Document]:\n        relevant_docs = []\n        for title, content in self.context_dict.items():\n            if any(kw in content.lower() for kw in query.lower().split()):\n                relevant_docs.append(\n                    Document(page_content=content, metadata={"Source": title})\n                )\n        return relevant_docs\n\n    async def _aget_relevant_documents(self, query: str, run_manager=None) -> List[Document]:\n        return self._get_relevant_documents(query, run_manager)\n\n# 2. user keyword + properly configu

In [ ]:
import gradio as gr
from typing import List, Dict
from langchain.schema import Document
from langchain_core.retrievers import BaseRetriever
from langchain.retrievers import EnsembleRetriever
from langchain.chains import ConversationalRetrievalChain
from langchain_core.callbacks import StdOutCallbackHandler
from langchain_community.chat_models import ChatOpenAI

# ========================
# 1. Kết nối LLM nội bộ
# ========================
llm = ChatOpenAI(
    api_key="lm",
    base_url="http://192.168.0.229:1234/v1",
    temperature=0.4,
    model_name=MODEL
)

# ========================
# 2. Dữ liệu mẫu trong context
# ========================
context = {
    "python": "Python is a programming language...",
    "ai": "Artificial Intelligence is ..."
}

# ========================
# 3. Keyword retriever
# ========================
class KeywordRetriever(BaseRetriever):
    context_dict: Dict[str, str]

    def _get_relevant_documents(self, query: str, run_manager=None) -> List[Document]:
        relevant_docs = []
        for title, content in self.context_dict.items():
            if any(kw in content.lower() for kw in query.lower().split()):
                relevant_docs.append(
                    Document(page_content=content, metadata={"Source": title})
                )
        return relevant_docs

    async def _aget_relevant_documents(self, query: str, run_manager=None) -> List[Document]:
        return self._get_relevant_documents(query, run_manager)

keyword_retriever = KeywordRetriever(context_dict=context)

# ========================
# 4. Vector retriever (ví dụ)
# ========================
# Giả sử bạn đã có vectorstore
# from your_vector_module import vectorstore
# vectorstore = ...
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# ========================
# 5. Hybrid retriever
# ========================
hybrid_retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, vector_retriever],
    weights=[0.5, 0.5],
)

# ========================
# 6. Conversation chain
# ========================
# Giả sử bạn có memory
# from your_memory_module import memory
# memory = ...
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=hybrid_retriever,
    memory=memory,
    callbacks=[StdOutCallbackHandler()]
)

# ========================
# 7. Bộ phân loại câu hỏi
# ========================
def classify_query(query: str) -> str:
    prompt = f"""
    Bạn là bộ phân loại câu hỏi.
    Phân loại câu hỏi sau thành một trong 2 loại:
    - "smalltalk": nếu là chào hỏi, trò chuyện phiếm, hỏi thời tiết, cảm xúc, ăn uống...
    - "knowledge": nếu là câu hỏi cần tìm thông tin từ cơ sở dữ liệu nội bộ.

    Chỉ trả lời đúng 1 từ: smalltalk hoặc knowledge.
    Câu hỏi: {query}
    """
    result = llm.invoke(prompt)
    return result.content.strip().lower()

# ========================
# 8. Hàm router xử lý câu hỏi
# ========================
def router(message, history):
    category = classify_query(message)
    if category == "smalltalk":
        return llm.invoke(message).content
    else:
        result = conversation_chain.invoke({"question": message})
        return result['answer']

# ========================
# 9. Gradio ChatInterface
# ========================
demo = gr.ChatInterface(
    fn=lambda messages, history: {"role": "assistant", "content": router(messages[-1]["content"], history)},
    type="messages",  # 🔹 thêm dòng này để dùng format mới, tránh cảnh báo
    title="Hybrid Chatbot with Router",
    description="Trả lời smalltalk trực tiếp, tìm kiếm kiến thức qua retriever khi cần.",
    theme="soft"
)


# if __name__ == "__main__":
#     demo.launch(server_name="0.0.0.0", server_port=7860)


/home/nguyenhaubt/anaconda3/envs/llm/lib/python3.10/site-packages/gradio/chat_interface.py:345: UserWarning:

The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.



In [ ]:
import gradio as gr
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result['answer'] #, result['source_documents'], history + [(message, result['answer'])]
    

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)